In [1]:
import folium
m=folium.Map(location=[48.85,2.4], zoom_start=11)

In [2]:
#a function to plot arrows manually
def arrow_points(lat1, long1, lat2, long2):
    sides_scale = 0.001
    sides_angle = 25
    
    if lat2!=lat1:
        tan=(long2-long1)/(lat2-lat1)
        if long2>=long1 and lat2>=lat1:
            heading=math.degrees(math.atan(tan))
        if long2<long1 and lat2>=lat1:
            heading=math.degrees(math.atan(tan))
        if lat2<lat1 and long2<long1:
            heading=180+math.degrees(math.atan(tan))
        if long2>=long1 and lat2<lat1:
            heading=180+math.degrees(math.atan(tan))      
    else:
        if long2>=long1:
            heading=90
        else:
            heading=270

    latA= lat1
    longA = long1
    
    latB= lat2
    longB = long2

    latC = sides_scale * math.cos(math.radians(heading + 180 - sides_angle)) + latB
    longC = sides_scale * math.sin(math.radians(heading + 180 - sides_angle)) + longB

    latD = sides_scale * math.cos(math.radians(heading + 180 + sides_angle)) + latB
    longD = sides_scale * math.sin(math.radians(heading + 180 + sides_angle)) + longB

    pointA = (latA, longA)
    pointB = (latB, longB)
    pointC = (latC, longC)
    pointD = (latD, longD)

    point = [pointA, pointB, pointC, pointD, pointB]
    return point

In [15]:
#import the flows and their coordinates
arr_cod_list=[]
with open('insta_we_arr_cod800403_Paris.txt','r') as f:
    flag = int(f.readline().strip('\n'))
    for line in f.readlines():
        line=line.strip('\n')
        line=line.split()
        cod1=[float(line[0]),float(line[1])]
        cod2=[float(line[2]),float(line[3])]
        x=float(line[4])
        nt1=int(line[5])
        arr_cod_list.append([cod1,cod2,x,nt1])

In [16]:
flag

1

In [17]:
def linear_map(a1,b1,a2,b2,x):
    #map from a1, b1 to a2 ,b2
    x=max(a1,x)
    x=min(b1,x)
    return (x-a1)*(b2-a2)/(b1-a1)+a2

In [18]:
#plot flows
import math
from collections import defaultdict
count=0
count2=0
evp_dict=defaultdict(int)
for arr_cod in arr_cod_list:
    x=arr_cod[2]
    if arr_cod[0]!=arr_cod[1]:
        if x>=500000:        
            count+=1
            lat1, long1=arr_cod[0][1],arr_cod[0][0]
            lat2, long2=arr_cod[1][1],arr_cod[1][0]
            r=255
            g=150-linear_map(500000,5000000,0,150,x)
            b=g
            color1= f"RGB({r},{g},{b})"
            points=arrow_points(lat1,long1,lat2,long2)
            folium.PolyLine(locations=points, color=color1,opacity=0.8).add_to(m)
    else:
        if x>1000:
            count2+=1
            nt=arr_cod[3]
            evp_dict[nt]=x
print(count,count2)

390 178


In [19]:
#plot the grids and evaporations
import branca.colormap as cm
if flag==1:
    linear = cm.LinearColormap(['white','green'],    #green means evporation
                           vmin=1000, vmax=800000)
else:
    linear = cm.LinearColormap(['white','red'],     ##red means increasement
                           vmin=1000, vmax=800000)
outline = 'Paris_c10.geojson'
folium.GeoJson(outline,style_function=lambda features: {
        'stroke': False,
        'fillColor': linear(evp_dict[features['properties']['new_id']]),
        "color": "black",
        "fillOpacity": 0.35,
        "weight": 0.5
    }).add_to(m)

In [20]:
linear(30)

'#ffffffff'

In [21]:
m

In [10]:
m.save("name_1.html")